<a href="https://colab.research.google.com/github/mudogruer/LLMs/blob/main/Fine_Tune_Mistral8x7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U transformers[torch]=='4.36.2' bitsandbytes peft datasets

In [ ]:
!pip install -q transformers[torch]=='4.36.2'

In [ ]:
import transformers
transformers.__version__

'4.36.2'

In [ ]:
!pip install -q trl

In [ ]:
from transformers import AutoTokenizer

In [ ]:
base_model = "mistralai/Mixtral-8x7B-v0.1"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    padding="max_length",
    truncation=True,
    padding_side = "right",
    add_eos_token = True,
    max_length=512,
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16,

)

In [ ]:

from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

In [ ]:
dataset_name = "databricks/databricks-dolly-15k"

In [ ]:
train_dataset = load_dataset(dataset_name, split="train[0:800]")
eval_dataset = load_dataset(dataset_name, split="train[800:1000]")

In [ ]:
train_dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 800
})

In [ ]:
train_dataset.to_pandas()

,instruction,context,response,category
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...,closed_qa
1,Which is a species of fish? Tope or Rope,,Tope,classification
2,Why can camels survive for long without water?,,Camels use the fat in their humps to keep them...,open_qa
3,"Alice's parents have three daughters: Amy, Jes...",,The name of the third daughter is Alice,open_qa
4,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981.",closed_qa
...,...,...,...,...
795,Who is the founder of the Communist Party?,,Lenin,open_qa
796,What is gardening?,Gardening is the practice of growing and culti...,Gardening is laying out and caring for a plot ...,information_extraction
797,What are your thoughts of Michael Jackson as a...,,Michael Jackson is acclaimed as the greatest p...,creative_writing
798,What is the largest pollutant?,,Carbon dioxide (CO2) - a greenhouse gas emitte...,general_qa


In [ ]:
train_dataset.to_pandas().dtypes

instruction    object
context        object
response       object
category       object
dtype: object

In [ ]:
train_dataset.to_pandas().value_counts("category")

category
open_qa                   202
general_qa                132
classification            111
brainstorming              95
closed_qa                  90
information_extraction     68
summarization              63
creative_writing           39
dtype: int64

In [ ]:
def generate_prompt(sample):
    full_prompt = f"""<s>[INST]{sample['instruction']}
    {f"Here is some context:{sample['context']}" if len(sample['context'])>0 else None}
    [/INST]{sample['response']}</s>
    """
    return {"text": full_prompt}

In [ ]:
train_dataset[0]

{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa'}

In [ ]:
generate_prompt(train_dataset[0])

{'text': "<s>[INST]When did Virgin Australia start operating?\n    Here is some context:Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.\n    [/INST]Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.</s>\n    "}

In [ ]:
generated_train_dataset = train_dataset.map(generate_prompt,
                                            remove_columns = list(train_dataset.features))
generated_val_dataset = eval_dataset.map(generate_prompt,
                                            remove_columns = list(train_dataset.features))


In [ ]:
generated_train_dataset

Dataset({
    features: ['text'],
    num_rows: 800
})

In [ ]:
#parameter efficient fine tuning
from peft import prepare_model_for_kbit_training

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"trainable params:{trainable_params}|| all params:{all_params} || trainable%: {100*trainable_params/all_params}")


In [ ]:
from peft import LoraConfig, get_peft_model

In [ ]:
lora_config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    target_modules =[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_proj",

    ],
    bias = "none",
    lora_dropout = 0.05,
    task_type = "CASUAL_LM"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
print_trainable_parameters(model)

trainable params:6815744|| all params:23489417216 || trainable%: 0.029016232873403956


In [ ]:
model

PeftModel(
  (base_model): LoraModel(
    (model): PeftModel(
      (base_model): LoraModel(
        (model): PeftModel(
          (base_model): LoraModel(
            (model): PeftModel(
              (base_model): LoraModel(
                (model): MixtralForCausalLM(
                  (model): MixtralModel(
                    (embed_tokens): Embedding(32000, 4096)
                    (layers): ModuleList(
                      (0-31): 32 x MixtralDecoderLayer(
                        (self_attn): MixtralAttention(
                          (q_proj): lora.Linear4bit(
                            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                            (lora_dropout): ModuleDict(
                              (default): Dropout(p=0.05, inplace=False)
                            )
                            (lora_A): ModuleDict(
                              (default): Linear(in_features=4096, out_features=8, bias=False)
                  

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_arguments = TrainingArguments(
    output_dir ="./results",
    num_train_epochs = 1,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 1,
    optim="paged_adamw_32bit",
    save_strategy ="steps",
    save_steps = 25,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    max_steps = 20,
    evaluation_strategy = "steps",
    eval_steps = 25,
    do_eval = True,
    report_to = "none",
    remove_unused_columns=False

)

In [ ]:
from trl import SFTTrainer

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    args = training_arguments,
    train_dataset = generated_train_dataset,
    eval_dataset = generated_val_dataset,
    peft_config = lora_config,
    dataset_text_field = "text",
    packing=True

)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
model.config.use_cache = False

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=20, training_loss=1.5922304153442384, metrics={'train_runtime': 425.2482, 'train_samples_per_second': 0.188, 'train_steps_per_second': 0.047, 'total_flos': 2.289428223492096e+16, 'train_loss': 1.5922304153442384, 'epoch': 0.49})

In [ ]:
my_finetuned_model = "mistral-8x7b-dolly"

In [ ]:
trainer.model.push_to_hub(my_finetuned_model)

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mudogruer/mistral-8x7b-dolly/commit/5b14c24010686ec6421ea60e8f52b53340247e0b', commit_message='Upload model', commit_description='', oid='5b14c24010686ec6421ea60e8f52b53340247e0b', pr_url=None, pr_revision=None, pr_num=None)